<a href="https://colab.research.google.com/github/smiling2727/AcaniChats/blob/master/NovelScraper_52shuku_consolidated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Input Parameters

In [1]:
list_url = 'https://www.52shuku.vip/bl/hs93.html'   # @param {type:"string"}
book_name = '[HP]我将为你送葬_麦野凉【完结】'  # @param {type:"string"}

## Install Dependencies

In [2]:
!apt update
!apt install -y chromium-chromedriver
!pip install selenium


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,105 kB]
G

## Scraping Functions


In [3]:
from bs4 import BeautifulSoup
import requests

def fetch_chapter_urls(list_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1"
    }

    response = requests.get(list_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <a> tags within <ul class="list clearfix">
    chapter_links = []
    ul_tag = soup.find("ul", class_="list clearfix")

    if ul_tag:
        for li_tag in ul_tag.find_all("li", class_="mulu"):
            a_tag = li_tag.find("a")
            if a_tag and "href" in a_tag.attrs:
                chapter_links.append(a_tag["href"])

    return chapter_links


In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

def fetch_chapter_urls(list_url):
    # Set up Selenium options for Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    # Start Chrome with Selenium
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(list_url)

    # Wait for the page to load completely
    time.sleep(2)  # Adjust as needed for the site's load time

    # Get the page source and parse it with BeautifulSoup
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Close the Selenium driver after fetching the content
    driver.quit()

    # Find all <a> tags within <ul class="list clearfix">
    chapter_links = []
    ul_tag = soup.find("ul", class_="list clearfix")

    if ul_tag:
        for li_tag in ul_tag.find_all("li", class_="mulu"):
            a_tag = li_tag.find("a")
            if a_tag and "href" in a_tag.attrs:
                chapter_links.append(a_tag["href"])

    return chapter_links

# Example usage
# list_url = "https://www.52shuku.vip/bl/20_b/bjV67.html"  # Replace with the actual list URL
# chapter_urls = fetch_chapter_urls(list_url)
# print(chapter_urls)

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

def scrape_chapter_content(url):
    # Set up Selenium options for Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")

    # Start Chrome with Selenium
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    # Wait for the page to load completely
    time.sleep(2)  # Adjust as needed for the site’s load time

    # Get the page source
    page_source = driver.page_source

    # Parse with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    article_content = soup.find("article", {"class": "article-content", "id": "nr1"})

    # Ensure Selenium driver is closed after scraping
    driver.quit()

    if not article_content:
        print(f"Page {url} is missing the expected content.")
        return ""

    # Extract paragraphs
    paragraphs = [
        p.get_text(strip=True)
        for p in article_content.find_all("p")
        if 'readinline' not in p.get("class", [])
    ]

    content = "\n".join(paragraphs)
    print("Scraping complete for chapter.")
    return content

# Example usage
# url = "https://www.52shuku.vip/bl/20_b/bjV67_2.html"  # Replace with the actual URL
# chapter_content = scrape_chapter_content(url)
# print(chapter_content)


In [6]:
def scrape_chapter_content_with_page_changes(url):
    content = ""
    page_number = 1  # Start with the main page
    last_page_content = None  # To store the content of the previous page

    while True:
        # Append page number postfix if not the main page
        page_url = url if page_number == 1 else f"{url.rsplit('.', 1)[0]}_{page_number}.html"

        print(f"Scraping page: {page_url}")

        try:

            # Join this page's paragraphs into a single string
            page_text = scrape_chapter_content(url)

            # Check for duplicate content
            if page_text == last_page_content:
                print(f"Duplicate content detected on {page_url}. Ending pagination.")
                break  # Stop if the current page's content matches the previous page's content

            # Append this page's content to the overall content
            content += page_text + "\n"
            last_page_content = page_text  # Update the last_page_content

            print(f"Successfully scraped {len(content)} characters from {page_url}")

            page_number += 1  # Move to the next page

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {page_url}: {e}")
            break

    return content

# Example usage
# url = "https://www.52shuku.vip/bl/20_b/bjV67_2.html"  # Replace with the actual URL
# chapter_content = scrape_chapter_content_with_page_changes(url)
# print(chapter_content)

## Run Scraper + Output to txt file

In [7]:

# Run Scraper
book_txt = book_name + '.txt'

chapter_urls = fetch_chapter_urls(list_url)
print(chapter_urls)

import time
new_content = ""
for index, href in enumerate(chapter_urls, start=1):
    # url = base_url + href
    url = href
    print("url: ", url)
    try:
        # Call the function and get content
        content = scrape_chapter_content_with_page_changes(url)

        # Print the index with the content
        # print(f"{index}\n{content}\n")
        print(f"Content from {url}: chapter {index}", ", total length:", len(new_content))
        new_content += f"Chapter {index}\n{content}\n\n"


    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")


['https://www.52shuku.vip/bl/hs93_2.html', 'https://www.52shuku.vip/bl/hs93_3.html', 'https://www.52shuku.vip/bl/hs93_4.html', 'https://www.52shuku.vip/bl/hs93_5.html', 'https://www.52shuku.vip/bl/hs93_6.html', 'https://www.52shuku.vip/bl/hs93_7.html', 'https://www.52shuku.vip/bl/hs93_8.html', 'https://www.52shuku.vip/bl/hs93_9.html', 'https://www.52shuku.vip/bl/hs93_10.html']
url:  https://www.52shuku.vip/bl/hs93_2.html
Scraping page: https://www.52shuku.vip/bl/hs93_2.html
Scraping complete for chapter.
Successfully scraped 2032 characters from https://www.52shuku.vip/bl/hs93_2.html
Scraping page: https://www.52shuku.vip/bl/hs93_2_2.html
Scraping complete for chapter.
Duplicate content detected on https://www.52shuku.vip/bl/hs93_2_2.html. Ending pagination.
Content from https://www.52shuku.vip/bl/hs93_2.html: chapter 1 , total length: 0
url:  https://www.52shuku.vip/bl/hs93_3.html
Scraping page: https://www.52shuku.vip/bl/hs93_3.html
Scraping complete for chapter.
Successfully scraped

In [8]:
# Output to a file
from google.colab import files
with open(book_txt, "w", encoding="utf-8") as file:
    file.write(new_content)
files.download(book_txt)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>